In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pandas as pd
import duckdb
import numpy as np
import altair as alt
import json
from urllib.request import urlopen
import time
import os
import plotly.express as px

In [2]:
poet = pd.read_csv('../../CCGIV/datasets/WomenWriting/poet.csv')
poem = pd.read_csv('../../CCGIV/datasets/WomenWriting/poem.csv')
poempoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/poempoetlinks.csv')
subwork = pd.read_csv('../../CCGIV/datasets/WomenWriting/subwork.csv')
subworkpoemlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/subworkpoemlinks.csv')
subworkpoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/subworkpoetlinks.csv')
work = pd.read_csv('../../CCGIV/datasets/WomenWriting/work.csv')
workpoemlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/workpoemlinks.csv')
workpoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/workpoetlinks.csv')
poetregionlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/poetregionlinks.csv')

In [3]:
work = work[work['worktype']=='別集']
work

,workID,AlephSysNo,Owner,OwnerCallNo,OwnerSysNo,TitleHZ,TitlePY,UniformTitle,VariantTitle,PubStatement,...,Summary,FormattedContents,Notes,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs,worktype
0,9,2185809,HYC,5490/4704,8.014468e+06,詠雪樓稿﹕五卷，卷首一卷，附一卷(清甘立媃撰),"Yong xue lou gao: 5 juan, juan shou 1 juan, fu...",NaN,NaN,清道光二十三年（1843）徐心田半偈齋刻本。,...,NaN,"卷1-4. 詠雲樓詩存 -- 卷5. 詠雪樓詩餘, 附歸舟圖詩微, 墓銘等.",附歸舟圖詩，又像贊、墓銘22頁。,前有宋鎔序、道光二十年（1840）王若閎序、嘉慶二十一年（1816）甘立媃自序。詩餘，前有甘...,末有道光二十三年徐心田跋。,NaN,NaN,NaN,NaN,別集
1,10,2000135,HYC,5517/8531,8.014483e+06,繡佛樓詩稿﹕二卷(清錢守璞撰),"Xiu fo lou shi gao: 2 juan (by Qian Shoupu, Qi...",NaN,NaN,清同治八年（1869）刻本。,...,NaN,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,NaN,"何褘序,錢守璞自序",NaN,NaN,NaN,NaN,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
2,11,1998985,HYC,5526/7254,8.014487e+06,曇花閣詩鈔﹕三集(清劉慧娟撰),"Tan hua ge shi chao: 3 ji (by Liu Huijuan, Qin...",NaN,NaN,清光緒十六年（1890）刻本。,...,NaN,計四種：治家恒言、試律、繡餘小草、餘生恨草。序等13頁。治家恒言、試律47頁。繡餘小草37頁...,NaN,"前有光緒十六年(1890)戴鴻慈序,光緒六年（1880）梁煦南序、光緒六年劉慧娟自序。",NaN,NaN,第一冊有扉頁，刻「曇花閣詩鈔初集。周雲瑑先生鑒定。湘舲女史著」。第二冊有扉頁，刻「曇花閣詩鈔...,NaN,《歷代婦女著作考》P.719著錄。,別集
3,12,1998812,HYC,5528/2341,8.014490e+06,佩秋閣遺稿﹕二卷﹐詞稿﹕一卷﹐文稿﹕一卷(清吳茝撰),"Pei qiu ge yi gao: 2 juan, ci gao: 1 juan, wen...",NaN,NaN,清光緒十四年（1888）刻本。,...,NaN,詩稿 2卷 -- 詞稿 1卷 -- 駢文稾 1卷.,序等16頁。詩稿卷上21頁。詩稿卷下21頁。詞稿8頁。文稿7頁。,前有同治五年（1866）許賡颺序、同治十年（1871）馮桂芬序、光緒元年秦雲序、光緒元年汪鶴...,末有汪鍾霖跋。,NaN,題「吳縣吳茝珮纕」。有扉頁，刻「佩秋閣遺稿。潘鍾瑞題。光緒元年乙亥九月開琱」。,NaN,《歷代婦女著作考》P.312著錄。,別集
4,13,1999271,HYC,T5493/3265,8.103586e+06,繡餘吟﹕六卷﹐附詩餘一卷(清馮思慧撰),"Xiu yu yin: 6 juan, fu shi yu 1 juan (by Feng ...",NaN,NaN,清乾隆二十九年（1764）刻本,...,NaN,NaN,題「鑑湖馮思慧睿之稿」。有扉頁，刻「繡餘吟」。卷一五言律詩四十四首，12頁。卷二七言律詩四十...,前有乾隆二十九年劉秉恬序。,NaN,NaN,NaN,NaN,《歷代婦女著作考》P.654著錄。,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,466,0,CUKH,NaN,NaN,"明秋館集:詩一卷, 詞一卷(清裘凌仙撰)","Ming qiu guan ji: shi 1 juan, ci 1 juan (by Qi...",NaN,NaN,民國排印本,...,NaN,NaN,"《歷代婦女著作考》著錄:《明秋館集》四卷(1913年排印本), 前有古裝小影，樊增祥、朱瑞序...",NaN,"朱瑞麟, 秦聯元,秦聯奎,秦飛卿",NaN,NaN,NaN,《歷代婦女著作考》第693頁著錄,別集
427,469,0,CUKH,PL2765.U238 A6 1910z,NaN,亭秋館詩鈔:六卷附錄一卷(清許禧身撰),Ting qiu guan shi chao: 6 juan fu lu 1 juan (b...,NaN,NaN,NaN,...,NaN,NaN,《歷代婦女著作考》錄《亭秋館詩鈔》為十卷，1912年刊於京師。然其每卷卷名與本集所見略有出入...,"陳夔龍, 俞陛雲",NaN,NaN,NaN,NaN,《歷代婦女著作考》567頁,別集
428,470,0,CUKH,NaN,NaN,望月軒詩詞集稿:二卷(清沈英撰),Wang yue xuan shi ci ji gao: 2 juan (By Shen Y...,NaN,NaN,廣東省城龍藏街萃經堂刊本,...,NaN,NaN,NaN,杜學謙,NaN,NaN,NaN,NaN,《歷代婦女著作考》第364頁著錄,別集
429,471,0,CUKH,NaN,NaN,儀光閣雜著:一卷(清武淑撰),"Yi guang ge za zhu: 1 juan (By Wu Shu, Qing dy...",NaN,NaN,西安排印本,...,NaN,NaN,《歷代婦女著作考》稱：此集附於其夫呂申華潔館集後。,武澂,NaN,NaN,NaN,NaN,《歷代婦女著作考》第399頁著錄,別集


In [4]:
workpoetlinks = workpoetlinks.merge(work, on='workID', how='inner')
workpoetlinks

,wcID,workID,poetID,role,date,AlephSysNo,Owner,OwnerCallNo,OwnerSysNo,TitleHZ,...,Summary,FormattedContents,Notes,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs,worktype
0,83,53,398,序作者,1891,1998534,HYC,5526/6434,8.014486e+06,紉蘭室詩鈔﹕三卷(清嚴永華撰),...,NaN,序14頁。卷一14頁。卷二12頁。卷三12頁。,NaN,前有光緒十七年張之萬序、龔易圖序、嚴辰序、光緒十五年（1889）朱福詵序。,NaN,NaN,題「桐鄉嚴永華少藍」。有扉頁，刻「紉蘭室詩鈔。寶熙題」。,NaN,《歷代婦女著作考》P.792著錄。,別集
1,17,53,149,主要作者,NaN,1998534,HYC,5526/6434,8.014486e+06,紉蘭室詩鈔﹕三卷(清嚴永華撰),...,NaN,序14頁。卷一14頁。卷二12頁。卷三12頁。,NaN,前有光緒十七年張之萬序、龔易圖序、嚴辰序、光緒十五年（1889）朱福詵序。,NaN,NaN,題「桐鄉嚴永華少藍」。有扉頁，刻「紉蘭室詩鈔。寶熙題」。,NaN,《歷代婦女著作考》P.792著錄。,別集
2,82,53,396,序作者,1891,1998534,HYC,5526/6434,8.014486e+06,紉蘭室詩鈔﹕三卷(清嚴永華撰),...,NaN,序14頁。卷一14頁。卷二12頁。卷三12頁。,NaN,前有光緒十七年張之萬序、龔易圖序、嚴辰序、光緒十五年（1889）朱福詵序。,NaN,NaN,題「桐鄉嚴永華少藍」。有扉頁，刻「紉蘭室詩鈔。寶熙題」。,NaN,《歷代婦女著作考》P.792著錄。,別集
3,19,53,397,序作者,1891,1998534,HYC,5526/6434,8.014486e+06,紉蘭室詩鈔﹕三卷(清嚴永華撰),...,NaN,序14頁。卷一14頁。卷二12頁。卷三12頁。,NaN,前有光緒十七年張之萬序、龔易圖序、嚴辰序、光緒十五年（1889）朱福詵序。,NaN,NaN,題「桐鄉嚴永華少藍」。有扉頁，刻「紉蘭室詩鈔。寶熙題」。,NaN,《歷代婦女著作考》P.792著錄。,別集
4,612,41,140,主要作者,NaN,2189788,HYC,5431/2326,7.238565e+06,同心梔子圖續編﹕一卷(清應瑩撰),...,NaN,序等11頁。續編24頁。,與《徐烈婦詩鈔》同函。,前有應瑩致吳晉齋啟。咸豐元年（1851）應瑩序、同治十三年（1874）俞樾序。,末有咸豐元年徐雨民跋。,題「陳鳳巢 齋先生鑒定、永康應瑩菉園著」。,NaN,NaN,《歷代婦女著作考》未著錄。,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,3712,473,8459,題辭,NaN,0,NLC,26390,3.120012e+11,"澹仙詩鈔四卷,詞鈔四卷,賦鈔一卷,文鈔一卷(清熊璉撰)",...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集
3026,3713,473,7547,跋作者,NaN,0,NLC,26390,3.120012e+11,"澹仙詩鈔四卷,詞鈔四卷,賦鈔一卷,文鈔一卷(清熊璉撰)",...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集
3027,3714,477,4624,跋作者,NaN,0,CUKH,NaN,NaN,山青雲白軒詩集:二卷(清傅宛撰),...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集
3028,3715,477,8460,其他作者,NaN,0,CUKH,NaN,NaN,山青雲白軒詩集:二卷(清傅宛撰),...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集


In [5]:
workpoemlinks = workpoemlinks.merge(work, on='workID', how='inner')
workpoemlinks

,wpID,workID,poemID,AlephSysNo,Owner,OwnerCallNo,OwnerSysNo,TitleHZ,TitlePY,UniformTitle,...,Summary,FormattedContents,Notes,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs,worktype
0,22652,45,22990,1999592,HYC,5508/2440,8.014472e+06,楚畹閣集﹕十二卷(清季蘭韻撰),"Chu wan ge ji: 12 juan (by Ji Lanyun, Qing dyn...",NaN,...,NaN,跋2頁。卷一13頁。卷二19頁。卷三16頁。卷四17頁。卷五19頁。卷六15頁。卷七16頁。...,NaN,無序。,末有道光二十七年姚福增跋。,NaN,題「常熟季蘭韻湘娟」。,NaN,《歷代婦女著作考》P.390著錄。,別集
1,54807,169,54938,0,SYSU,18506,NaN,息存室吟稿初集一卷續集一卷:二卷(清杭溫如撰),Xi cun shi yin gao chu ji yi juan xu ji yi jua...,NaN,...,NaN,NaN,嘉慶13年戊辰原刻續集嘉慶22年丁丑(1817),孟元泰、許麗京、胡啓虞、趙元中,NaN,NaN,NaN,NaN,歷代婦女著作考 395頁,別集
2,54844,170,54975,0,SYSU,03550,NaN,聽秋軒詩集:六卷(清駱綺蘭撰),"Ting qiu xuan shi ji: 6 juan (by Luo Qilan, Qi...",NaN,...,NaN,NaN,NaN,袁枚、王文治、曾燠,NaN,NaN,NaN,NaN,歷代婦女著作考 761頁,別集
3,54839,170,54970,0,SYSU,03550,NaN,聽秋軒詩集:六卷(清駱綺蘭撰),"Ting qiu xuan shi ji: 6 juan (by Luo Qilan, Qi...",NaN,...,NaN,NaN,NaN,袁枚、王文治、曾燠,NaN,NaN,NaN,NaN,歷代婦女著作考 761頁,別集
4,54874,170,55006,0,SYSU,03550,NaN,聽秋軒詩集:六卷(清駱綺蘭撰),"Ting qiu xuan shi ji: 6 juan (by Luo Qilan, Qi...",NaN,...,NaN,NaN,NaN,袁枚、王文治、曾燠,NaN,NaN,NaN,NaN,歷代婦女著作考 761頁,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53992,96738,477,96835,0,CUKH,NaN,NaN,山青雲白軒詩集:二卷(清傅宛撰),Shan qing yun bai xuan shi ji: 2 juan (by Fu W...,NaN,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集
53993,96739,477,96836,0,CUKH,NaN,NaN,山青雲白軒詩集:二卷(清傅宛撰),Shan qing yun bai xuan shi ji: 2 juan (by Fu W...,NaN,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集
53994,96740,477,96837,0,CUKH,NaN,NaN,山青雲白軒詩集:二卷(清傅宛撰),Shan qing yun bai xuan shi ji: 2 juan (by Fu W...,NaN,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集
53995,96845,473,96942,0,NLC,26390,3.120012e+11,"澹仙詩鈔四卷,詞鈔四卷,賦鈔一卷,文鈔一卷(清熊璉撰)","Danxian shi chao 4 juan, ci chao 4 juan, fu ch...",NaN,...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集


In [6]:
poet = poet.merge(workpoetlinks, on='poetID',how='inner')
poet = poet.drop_duplicates(subset='poetID')

poet

,poetID,NameHZ,NamePY,HaoHZ,HaoPY,ZiHZ,ZiPY,OtherZiHaoHZ,OtherZiHaoPY,MaritalStatus,...,Summary,FormattedContents,Notes_y,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs,worktype
0,5,甘立媃,Gan Lirou,NaN,NaN,如玉,Ruyu,NaN,NaN,正室﹐寡婦,...,NaN,"卷1-4. 詠雲樓詩存 -- 卷5. 詠雪樓詩餘, 附歸舟圖詩微, 墓銘等.",附歸舟圖詩，又像贊、墓銘22頁。,前有宋鎔序、道光二十年（1840）王若閎序、嘉慶二十一年（1816）甘立媃自序。詩餘，前有甘...,末有道光二十三年徐心田跋。,NaN,NaN,NaN,NaN,別集
2,6,劉慧娟,Liu Huijuan,幻花女史,Huanhuanüshi,湘舲,Xiangling,NaN,NaN,正室﹐寡婦,...,NaN,計四種：治家恒言、試律、繡餘小草、餘生恨草。序等13頁。治家恒言、試律47頁。繡餘小草37頁...,NaN,"前有光緒十六年(1890)戴鴻慈序,光緒六年（1880）梁煦南序、光緒六年劉慧娟自序。",NaN,NaN,第一冊有扉頁，刻「曇花閣詩鈔初集。周雲瑑先生鑒定。湘舲女史著」。第二冊有扉頁，刻「曇花閣詩鈔...,NaN,《歷代婦女著作考》P.719著錄。,別集
4,8,馮思慧,Feng Sihui,NaN,NaN,睿之,Ruizhi,駱思慧,Luo Sihui,正室,...,NaN,NaN,題「鑑湖馮思慧睿之稿」。有扉頁，刻「繡餘吟」。卷一五言律詩四十四首，12頁。卷二七言律詩四十...,前有乾隆二十九年劉秉恬序。,NaN,NaN,NaN,NaN,《歷代婦女著作考》P.654著錄。,別集
5,11,屠鏡心,Tu Jingxin,掃花主人,Saohuazhuren,NaN,NaN,NaN,NaN,正室,...,NaN,序跋等27頁。上卷22頁。下卷23頁。,NaN,前有同治九年文廉序、張之縉序、同治六年（1867）耀曾序。戴嘉瑞、戴嘉玉、傅隱蘭、于懿、鄧渝題辭。,同治八年（1869）任珮瑛跋、同治八年萬秉跋、同治九年謙吉跋、任凱跋、同治三年（1864）任治跋。,NaN,題「陽羨屠鏡心掃花主人賸草」。鈐印有「小玲瓏精舍」、「□業書田」。,NaN,《歷代婦女著作考》P.627著錄。,別集
6,12,鄭蘭孫,Zheng Lansun,蘅洲,Hengzhou,娛清,Yuqing,NaN,NaN,正室,...,NaN,序、題辭、跋等27頁。詩集上16頁、詩集下25頁。詞集14頁。,●燕京图书馆又有複本一部，四冊。 ●燕京图书馆又有複本一部，四冊。 ●燕京图书馆又有複本一部...,前有俞樾撰「鄭孺人傳」。道光二十三年（1843）徐若洲序、咸豐十年（1860）錢士杓序、咸豐...,末有光緒元年許樾身跋。光緒元年徐琪附記。,NaN,有扉頁，刻「蓮因室詩詞集。光緒元年乙亥春彭玉麔署檢」。,NaN,《歷代婦女著作考》P.744著錄。,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,8457,于泗,Yu Si,NaN,NaN,秋渚,Qiuzhu,NaN,NaN,NaN,...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集
3026,8458,秦鼎雲,Qin Dingyun,NaN,NaN,玉齋,Yuzhai,NaN,NaN,NaN,...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集
3027,8459,朱雨田,Zhu Yutian,NaN,NaN,硯農,Yannong,NaN,NaN,NaN,...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集
3028,8460,俞彥文,Yu Yanwen,NaN,NaN,人蔚,Renwei,NaN,NaN,NaN,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集


In [7]:
poem = poem.merge(workpoemlinks, on='poemID',how='inner')
poem = poem.drop_duplicates(subset='poemID')

poem

,poemID,poetassubjectID,changheshuxinpoetID,tempID,TitleHZ_x,TitlePY_x,GenreHZ,GenrePY,Form,TunePatternSubtitle,...,Summary,FormattedContents,Notes_y,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs_y,worktype
0,4225,0,0,0,臺城路,Tai cheng lu,詞,ci,NaN,紅葉,...,NaN,詩稿 2卷 -- 詞稿 1卷 -- 駢文稾 1卷.,序等16頁。詩稿卷上21頁。詩稿卷下21頁。詞稿8頁。文稿7頁。,前有同治五年（1866）許賡颺序、同治十年（1871）馮桂芬序、光緒元年秦雲序、光緒元年汪鶴...,末有汪鍾霖跋。,NaN,題「吳縣吳茝珮纕」。有扉頁，刻「佩秋閣遺稿。潘鍾瑞題。光緒元年乙亥九月開琱」。,NaN,《歷代婦女著作考》P.312著錄。,別集
1,4220,0,0,0,送業師鶴巢夫子公車北上,Song ye shi Hechao fu zi gong che bei shang,詩,shi,七言律詩,NaN,...,NaN,詩稿 2卷 -- 詞稿 1卷 -- 駢文稾 1卷.,序等16頁。詩稿卷上21頁。詩稿卷下21頁。詞稿8頁。文稿7頁。,前有同治五年（1866）許賡颺序、同治十年（1871）馮桂芬序、光緒元年秦雲序、光緒元年汪鶴...,末有汪鍾霖跋。,NaN,題「吳縣吳茝珮纕」。有扉頁，刻「佩秋閣遺稿。潘鍾瑞題。光緒元年乙亥九月開琱」。,NaN,《歷代婦女著作考》P.312著錄。,別集
2,4205,5,0,0,歸舟安侍圖題辭,Gui zhou an shi tu ti ci,文﹕題辭,Wen: Ti ci,NaN,NaN,...,NaN,"卷1-4. 詠雲樓詩存 -- 卷5. 詠雪樓詩餘, 附歸舟圖詩微, 墓銘等.",附歸舟圖詩，又像贊、墓銘22頁。,前有宋鎔序、道光二十年（1840）王若閎序、嘉慶二十一年（1816）甘立媃自序。詩餘，前有甘...,末有道光二十三年徐心田跋。,NaN,NaN,NaN,NaN,別集
3,4206,5,0,0,前題[步韻],Qian ti [Bu yun],詩,shi,七言古詩,NaN,...,NaN,"卷1-4. 詠雲樓詩存 -- 卷5. 詠雪樓詩餘, 附歸舟圖詩微, 墓銘等.",附歸舟圖詩，又像贊、墓銘22頁。,前有宋鎔序、道光二十年（1840）王若閎序、嘉慶二十一年（1816）甘立媃自序。詩餘，前有甘...,末有道光二十三年徐心田跋。,NaN,NaN,NaN,NaN,別集
4,4221,0,0,0,寄荅陸元芬,Ji da Lu Yuanfen,詩,shi,七言古詩,NaN,...,NaN,詩稿 2卷 -- 詞稿 1卷 -- 駢文稾 1卷.,序等16頁。詩稿卷上21頁。詩稿卷下21頁。詞稿8頁。文稿7頁。,前有同治五年（1866）許賡颺序、同治十年（1871）馮桂芬序、光緒元年秦雲序、光緒元年汪鶴...,末有汪鍾霖跋。,NaN,題「吳縣吳茝珮纕」。有扉頁，刻「佩秋閣遺稿。潘鍾瑞題。光緒元年乙亥九月開琱」。,NaN,《歷代婦女著作考》P.312著錄。,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53992,96835,0,0,0,曡韻四贈迦陵,Die yun si zeng Jialing,詩,shi,七言律詩,NaN,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集
53993,96836,0,0,0,閱楊詠裳先生來杭偕彥兒兄弟遊湖詩書後(二首),Yue Yang Yongshang xian sheng lai Hang xie Yan...,詩,shi,七言律詩,NaN,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集
53994,96837,0,0,0,代彥兒贈鈕耕孫大令,Dai Yan er zeng niu geng Sun da ling,詩,shi,七言律詩,NaN,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集
53995,96942,0,0,0,襯頁,Chen ye,NaN,NaN,NaN,NaN,...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集


In [8]:
con = duckdb.connect()
con.register('poet', poet)
con.register('poem', poem)
con.register('poempoetlinks', poempoetlinks)
con.register('subwork', subwork)
con.register('subworkpoemlinks', subworkpoemlinks)
con.register('subworkpoetlinks', subworkpoetlinks)
con.register('work', work)
con.register('workpoemlinks', workpoemlinks)
con.register('workpoetlinks', workpoetlinks)
con.register('poetregionlinks', poetregionlinks)

In [9]:
ticiSQL = '''
SELECT workID,count(DISTINCT poetID) as ticicount
FROM workpoetlinks 
WHERE role IN ('題辭')
Group By workID
ORDER BY ticicount DESC
'''
tici = duckdb.query(ticiSQL).df()
tici.set_index('workID', inplace=True)

xuSQL = '''
SELECT workID,count(DISTINCT poetID) as xucount
FROM workpoetlinks 
WHERE role IN ('序作者')
Group By workID
ORDER BY xucount DESC
'''
xu = duckdb.query(xuSQL).df()
xu.set_index('workID', inplace=True)

baSQL = '''
SELECT workID,count(DISTINCT poetID) as bacount
FROM workpoetlinks 
WHERE role IN ('跋作者')
Group By workID
ORDER BY bacount DESC
'''
ba = duckdb.query(baSQL).df()
ba.set_index('workID', inplace=True)

from functools import reduce

TiciXuBa = reduce(lambda left, right: pd.merge(left, right, on='workID', how='outer'), [tici, xu, ba]).fillna(0)
TiciXuBa

includedWorkSQL = '''
SELECT workID,count(poemID) as includedcount
FROM workpoemlinks 
Group By workID
ORDER BY includedcount DESC
'''
includedWork = duckdb.query(includedWorkSQL).df()
includedWork.set_index('workID', inplace=True)

jiRawData = reduce(lambda left, right: pd.merge(left, right, on='workID', how='outer'), [tici, xu, ba, includedWork]).fillna(0)

ji_combine_normalize = jiRawData.copy()

all_counts = np.concatenate((ji_combine_normalize['ticicount'], ji_combine_normalize['xucount'],ji_combine_normalize['bacount']))

min_val = all_counts.min()
max_val = all_counts.max()
ji_combine_normalize['normalized_bacount'] = (ji_combine_normalize['bacount'] - min_val) / (max_val - min_val)
ji_combine_normalize['normalized_ticicount'] = (ji_combine_normalize['ticicount'] - min_val) / (max_val - min_val)
ji_combine_normalize['normalized_xucount'] = (ji_combine_normalize['xucount'] - min_val) / (max_val - min_val)

def min_max_normalize_ori(series):
    return (series - series.min()) / (series.max() - series.min())
# 收录的作品数单独归一化
if ji_combine_normalize['includedcount'].dtype in ['float64', 'int64']:  # 检查数据类型是否为数值类型
        ji_combine_normalize['includedcount'] = min_max_normalize_ori(ji_combine_normalize['includedcount'])

xuweight = 0.25
baweight = 0.25
ticiweight = 0.25
includedweight = 0.25

ji_combine = ji_combine_normalize.copy()

ji_combine['totalWeight'] = xuweight*ji_combine['normalized_xucount'] + baweight*ji_combine['normalized_bacount'] + ticiweight*ji_combine['normalized_ticicount'] + includedweight*ji_combine['includedcount']

ji_combine

,ticicount,xucount,bacount,includedcount,normalized_bacount,normalized_ticicount,normalized_xucount,totalWeight
workID,,,,,,,,
9,0.0,3.0,1.0,0.628517,0.008403,0.000000,0.025210,0.165533
10,2.0,2.0,0.0,0.256394,0.000000,0.016807,0.016807,0.072502
11,0.0,3.0,0.0,0.085678,0.000000,0.000000,0.025210,0.027722
12,1.0,5.0,1.0,0.078645,0.008403,0.008403,0.042017,0.034367
13,0.0,1.0,0.0,0.159207,0.000000,0.000000,0.008403,0.041903
...,...,...,...,...,...,...,...,...
476,1.0,4.0,0.0,0.248082,0.000000,0.008403,0.033613,0.072525
477,14.0,5.0,1.0,0.148338,0.008403,0.117647,0.042017,0.079101
478,0.0,0.0,0.0,0.129795,0.000000,0.000000,0.000000,0.032449


In [ ]:
PoetParticipateWorkSQL = '''SELECT poetID, COUNT(DISTINCT workID) AS participate_count
    FROM workpoetlinks
    GROUP BY poetID
    ORDER BY participate_count DESC'''

PoetParticipateWork = duckdb.query(PoetParticipateWorkSQL).df()
PoetParticipateWork.set_index('poetID', inplace=True)

xiangzanauthorSQL = '''
SELECT poempoetlinks.poetID,COUNT(DISTINCT poem.poemID) AS xiangzanauthor_count
FROM poem,poempoetlinks
WHERE poem.GenreHZ='文﹕像贊' AND poem.poemID = poempoetlinks.poemID
GROUP BY poempoetlinks.poetID
ORDER BY xiangzanauthor_count
'''
xiangzanauthor = duckdb.query(xiangzanauthorSQL).df()
xiangzanauthor.set_index('poetID', inplace=True)

